In [1]:
import arcpy
import pandas as pd
import os

In [2]:
#function to create a pandas df from feature class
def fcToPandasDF(fcobj, aAttributes):
    return (pd.DataFrame( arcpy.da.FeatureClassToNumPyArray(in_table = fcobj, field_names = aAttributes,  skip_nulls = False, null_value = -99999)))

In [3]:
#environment configuration
arcpy.env.workspace = r"D:\ONU\Processing\results_integration_updated.gdb"
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1927 UTM Zone 18N")
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

In [4]:
#original building data
buildings_ms = r"D:\ONU\Processing\HTI_Modeling_Pop_Inputs.gdb\MS_FPrints"
buildings_hotosm = r"D:\ONU\Processing\HTI_Modeling_Pop_Inputs.gdb\Hotosm_buildings"
admin_3 = r"D:\ONU\Processing\HTI_Modeling_Pop_Inputs.gdb\adm3_cnigs_ocha"
GDB_outputs = r"D:\ONU\Processing\results_integration_updated.gdb"

In [5]:
#selecting only buildings ms within admin3 borders
lyr_ms_ori = "lyr_ms_ori"
arcpy.management.MakeFeatureLayer(in_features=buildings_ms, out_layer=lyr_ms_ori, where_clause="")

lyr_admin3 = "lyr_admin3"
arcpy.management.MakeFeatureLayer(in_features=admin_3, out_layer=lyr_admin3, where_clause="")

selected_buildings_ms_ori = arcpy.management.SelectLayerByLocation(in_layer=[lyr_ms_ori], overlap_type="INTERSECT", select_features=lyr_admin3, search_distance="", selection_type="NEW_SELECTION", invert_spatial_relationship="")

buildings_to_include = os.path.join(GDB_outputs, "ms_hti_fprint_in_admin_v2")
arcpy.management.CopyFeatures(in_features=selected_buildings_ms_ori, out_feature_class=buildings_to_include)

buildings_to_include_multi = os.path.join(GDB_outputs, "ms_hti_fprint_in_admin_v2_multi")
arcpy.management.MultipartToSinglepart(buildings_to_include, buildings_to_include_multi)

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\ms_hti_fprint_in_admin_v2_multi'>

In [6]:
#selecting only buildings hotosm within admin3 borders
lyr_hotosm_ori = "lyr_hotosm_ori"
arcpy.management.MakeFeatureLayer(in_features=buildings_hotosm, out_layer=lyr_hotosm_ori, where_clause="")

lyr_admin3 = "lyr_admin3"
arcpy.management.MakeFeatureLayer(in_features=admin_3, out_layer=lyr_admin3, where_clause="")

selected_buildings_hotosm_ori = arcpy.management.SelectLayerByLocation(in_layer=[lyr_hotosm_ori], overlap_type="INTERSECT", select_features=lyr_admin3, search_distance="", selection_type="NEW_SELECTION", invert_spatial_relationship="")

buildings_to_include_hotosm = os.path.join(GDB_outputs, "hotosm_hti_fprint_in_admin_v2")
arcpy.management.CopyFeatures(in_features=selected_buildings_hotosm_ori, out_feature_class=buildings_to_include_hotosm)

buildings_to_include_hotosm_multi = os.path.join(GDB_outputs, "hotosm_hti_fprint_in_admin_v2_multi")
arcpy.management.MultipartToSinglepart(buildings_to_include_hotosm, buildings_to_include_hotosm_multi)

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\hotosm_hti_fprint_in_admin_v2_multi'>

In [7]:
buildings_ms_processed = buildings_to_include
buildings_ms_processed

'D:\\ONU\\Processing\\results_integration_updated.gdb\\ms_hti_fprint_in_admin_v2'

In [8]:
buff_general_ms_5m = os.path.join(GDB_outputs, "buff_general_bms")
arcpy.analysis.Buffer(in_features=buildings_ms_processed, out_feature_class=buff_general_ms_5m, buffer_distance_or_field="5 Meters", line_side="FULL", line_end_type="ROUND")

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\buff_general_bms'>

In [10]:
buildings_hotosm_processed = buildings_to_include_hotosm_multi
buildings_hotosm_processed

'D:\\ONU\\Processing\\results_integration_updated.gdb\\hotosm_hti_fprint_in_admin_v2_multi'

In [11]:
#managing entire country info
lyr_hotsom = "adm3_hotosm_b"
arcpy.management.MakeFeatureLayer(in_features=buildings_hotosm_processed, out_layer=lyr_hotsom, where_clause="")

lyr_ms_buff = "buff_ms_b"
arcpy.management.MakeFeatureLayer(in_features=buff_general_ms_5m, out_layer=lyr_ms_buff, where_clause="")

selected_buildings_hotosm = arcpy.management.SelectLayerByLocation(in_layer=[lyr_hotsom], overlap_type="INTERSECT", select_features=lyr_ms_buff, search_distance="", selection_type="NEW_SELECTION", invert_spatial_relationship="INVERT")

buildings_no_included = os.path.join(GDB_outputs, "no_included_general_bhotosm")
arcpy.management.CopyFeatures(in_features=selected_buildings_hotosm, out_feature_class=buildings_no_included)

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\no_included_general_bhotosm'>

In [12]:
#merging ms and hotosm info

merge_general_ms_hotosm = os.path.join(GDB_outputs, "merge_general_buildings_ms_osm")
arcpy.management.Merge([buildings_ms_processed, buildings_no_included], merge_general_ms_hotosm, "",'ADD_SOURCE_INFO')

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\merge_general_buildings_ms_osm'>

In [13]:
#creating buffer 5m to the previous merged layer

buff_merged_ms_hotosm_5m = os.path.join(GDB_outputs, "buff_merged_ms_hotosm_5m")
arcpy.analysis.Buffer(in_features=merge_general_ms_hotosm, out_feature_class=buff_merged_ms_hotosm_5m, buffer_distance_or_field="5 Meters", line_side="FULL", line_end_type="ROUND", dissolve_option="NONE", dissolve_field=[], method="PLANAR")

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\buff_merged_ms_hotosm_5m'>

In [14]:
#selecting survey points to include

survey_points_hhloc = r"D:\ONU\Processing\HTI_Modeling_Pop_Inputs.gdb\main_hhloc_clean_26718"

lyr_survey_points = "survey_points"
arcpy.management.MakeFeatureLayer(in_features=survey_points_hhloc, out_layer=lyr_survey_points, where_clause="")

lyr_ms_hotosm_buff = "buff_ms_hotosm_b"
arcpy.management.MakeFeatureLayer(in_features=buff_merged_ms_hotosm_5m, out_layer=lyr_ms_hotosm_buff, where_clause="")

selected_buildings_survey = arcpy.management.SelectLayerByLocation(in_layer=[lyr_survey_points], overlap_type="INTERSECT", select_features=lyr_ms_hotosm_buff, search_distance="", selection_type="NEW_SELECTION", invert_spatial_relationship="INVERT")

points_survey_no_included = os.path.join(GDB_outputs, "no_included_general_survey_points")
arcpy.management.CopyFeatures(in_features=selected_buildings_survey, out_feature_class=points_survey_no_included)


<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\no_included_general_survey_points'>

In [15]:
#converting merged builgings to points
points_general_ms_hotosm = os.path.join(GDB_outputs, "points_general_ms_hotosm")
arcpy.management.FeatureToPoint(merge_general_ms_hotosm, points_general_ms_hotosm, "INSIDE")

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\points_general_ms_hotosm'>

In [16]:
#merging points ms-osm with survey points

merge_general_ms_hotosm_survey = os.path.join(GDB_outputs, "merge_general_ms_hotosm_survey")
arcpy.management.Merge([points_general_ms_hotosm, points_survey_no_included], merge_general_ms_hotosm_survey, "",'ADD_SOURCE_INFO')

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\merge_general_ms_hotosm_survey'>

In [17]:
#excluding points ms-osm-survey that are whithin water bodies

water_b = r"D:\ONU\Processing\HTI_Modeling_Pop_Inputs.gdb\osm_water_bodies"

lyr_ms_osm_survey_p = "lyr_ms_osm_survey_p"
arcpy.management.MakeFeatureLayer(in_features=merge_general_ms_hotosm_survey, out_layer=lyr_ms_osm_survey_p, where_clause="")

lyr_water = "lyr_water_b"
arcpy.management.MakeFeatureLayer(in_features=water_b, out_layer=lyr_water, where_clause="")

selected_buildings_total = arcpy.management.SelectLayerByLocation(in_layer=[lyr_ms_osm_survey_p], overlap_type="INTERSECT", select_features=lyr_water, search_distance="", selection_type="NEW_SELECTION", invert_spatial_relationship="INVERT")

buildings_no_included_total = os.path.join(GDB_outputs, "final_to_include_total_no_water")
arcpy.management.CopyFeatures(in_features=selected_buildings_total, out_feature_class=buildings_no_included_total)

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\final_to_include_total_no_water'>

In [6]:
################
#generating raster from integrated points p1
buildings_no_included_total = os.path.join(GDB_outputs, "final_to_include_total_no_water")
lyr_merge_general_ms_hotosm_survey = "lyr_merge_general_ms_hotosm_survey"
arcpy.management.MakeFeatureLayer(in_features=buildings_no_included_total, out_layer=lyr_merge_general_ms_hotosm_survey, where_clause="")

lyr_admin_3 = "lyr_admin_3"
arcpy.management.MakeFeatureLayer(in_features=admin_3, out_layer=lyr_admin_3, where_clause="")

selected_buildings_adm3 = arcpy.management.SelectLayerByLocation(in_layer=[lyr_merge_general_ms_hotosm_survey], overlap_type="INTERSECT", select_features=lyr_admin_3, search_distance="", selection_type="NEW_SELECTION")

total_points_ms_hotosm_survey_adm3 = os.path.join(GDB_outputs, "total_points_ms_hotosm_survey_adm3")
arcpy.management.CopyFeatures(in_features=selected_buildings_adm3, out_feature_class=total_points_ms_hotosm_survey_adm3)

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\total_points_ms_hotosm_survey_adm3'>

In [7]:
#generating raster from integrated points p2

raster_points_ms_hotosm_survey = os.path.join(GDB_outputs, "raster_ms_hotosm_survey")
arcpy.conversion.PointToRaster(total_points_ms_hotosm_survey_adm3, "OBJECTID", raster_points_ms_hotosm_survey, "COUNT", "NONE", 100, "BUILD")

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\raster_ms_hotosm_survey'>

In [9]:
#creating new raster layer wih 0 value p1

diss_hti_adm3 = r"D:\ONU\Processing\HTI_Modeling_Pop_Inputs.gdb\diss_adm3_cnigs_ocha"
minbbox_hti_adm3 = os.path.join(GDB_outputs, "minbbox_hti_adm3")
arcpy.management.MinimumBoundingGeometry(diss_hti_adm3, minbbox_hti_adm3, "ENVELOPE", "NONE", None, "NO_MBG_FIELDS")

arcpy.management.AddField(minbbox_hti_adm3,"value", 'LONG')
arcpy.management.CalculateField(minbbox_hti_adm3, "value", 0, expression_type="PYTHON3", code_block="", field_type="LONG")

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\minbbox_hti_adm3'>

In [10]:
#creating new raster layer wih 0 value p2

raster_zero_envelope = os.path.join(GDB_outputs, "raster_zero_envelope")
arcpy.conversion.FeatureToRaster(minbbox_hti_adm3, "value", raster_zero_envelope, 100)

<Result 'D:\\ONU\\Processing\\results_integration_updated.gdb\\raster_zero_envelope'>

In [11]:
#applying map algebra

raster_sum_total_envelope = os.path.join(GDB_outputs, "raster_sum_total_envelope")
output_raster = arcpy.ia.RasterCalculator([raster_points_ms_hotosm_survey, raster_zero_envelope],
                                         ["x", "y"], "x+y")
output_raster.save(raster_sum_total_envelope)

In [12]:
raster_sum_total_envelope_2 = os.path.join(GDB_outputs, "raster_sum_total_envelope_2")
output_raster = arcpy.ia.RasterCalculator([raster_sum_total_envelope],
                                         ["x"], "Con(IsNull(x), 0, x)")
output_raster.save(raster_sum_total_envelope_2)

In [7]:
#cropping final raster

final_raster_buildings_cropped_hti = os.path.join(GDB_outputs, "final_raster_buildings_cropped_hti_")
raster_sum_total_envelope_2 = os.path.join(GDB_outputs, "raster_sum_total_envelope_2")
buff_diss_admin3 = r"D:\ONU\Processing\HTI_Modeling_Pop_Inputs.gdb\diss_adm3_cnigs_ocha_buff_100m"

out_raster = arcpy.sa.ExtractByMask(raster_sum_total_envelope_2, buff_diss_admin3)
out_raster.save(final_raster_buildings_cropped_hti)

In [ ]:

#function to calculate the builgings from osm to include in the ms dataset considering 5m buffer
#agregattion by admin level 3

def calculate_to_include_buildings(level3_df, buff_param_dist):
    
    adm3_ind = level3_df['OBJECTID'].tolist()

    for i in range(len(adm3_ind)):

        try:

            print("Processing " + str(adm3_ind[i]) + " ...")

            adm3_selected = os.path.join(GDB_outputs, "adm3_selected_" + str(adm3_ind[i]))
            arcpy.analysis.Select(admin_3, adm3_selected, "OBJECTID = " + str(adm3_ind[i]))

            inter_adm3_sel_ms = os.path.join(GDB_outputs, "inter_adm3_selected_bms" + str(adm3_ind[i]))
            arcpy.analysis.Intersect(in_features=[adm3_selected, buildings_ms_processed], out_feature_class=inter_adm3_sel_ms, join_attributes="ALL", cluster_tolerance="", output_type="INPUT")

            inter_adm3_sel_hotosm = os.path.join(GDB_outputs, "inter_adm3_selected_bhotosm" + str(adm3_ind[i]))
            arcpy.analysis.Intersect(in_features=[adm3_selected, buildings_hotosm_processed], out_feature_class=inter_adm3_sel_hotosm, join_attributes="ALL", cluster_tolerance="", output_type="INPUT")

            buff_inter_adm3_sel_ms = os.path.join(GDB_outputs, "buff_inter_adm3_selected_bms" + str(adm3_ind[i]))
            arcpy.analysis.Buffer(in_features=inter_adm3_sel_ms, out_feature_class=buff_inter_adm3_sel_ms, buffer_distance_or_field=buff_param_dist, line_side="FULL", line_end_type="ROUND", dissolve_option="NONE", dissolve_field=[], method="PLANAR")

            lyr_hotsom = "adm3_sel_hotosm_b"
            arcpy.management.MakeFeatureLayer(in_features=inter_adm3_sel_hotosm, out_layer=lyr_hotsom, where_clause="")

            lyr_ms_buff = "buff_adm3_sel_ms_b"
            arcpy.management.MakeFeatureLayer(in_features=buff_inter_adm3_sel_ms, out_layer=lyr_ms_buff, where_clause="")

            selected_buildings_hotosm = arcpy.management.SelectLayerByLocation(in_layer=[lyr_hotsom], overlap_type="INTERSECT", select_features=lyr_ms_buff, search_distance="", selection_type="NEW_SELECTION", invert_spatial_relationship="INVERT")

            buildings_no_included = os.path.join(GDB_outputs, "no_included_selected_bhotosm" + str(adm3_ind[i]))
            arcpy.management.CopyFeatures(in_features=selected_buildings_hotosm, out_feature_class=buildings_no_included)

            print("Done...")

        except:
            print("Error in adm3... " + str(adm3_ind[i]) + " ...")
    